# 3: Starting FHIST from spunup state in 1850

By default the FHIST starts from a default initial condition in 1979, but you can start from another year and use initial conditions coming from a previous run. 


<div class="alert alert-info">
<strong>Exercise: Customize your CAM history files</strong><br><br>
 
Create, configure, build and run a case called `fhist.1850` with the compset FHIST at the resolution `f19_f19_mg17` using the same history file output as in the control. 
- Use initialization datasets from a previous case. <br> For instance, use the data from the run: `b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001` at year `321`
- Start model in `1850`. 
- Run for 5 days. 

Check your solution:
- How can you check that there is a difference between the set up of this and your control?
- How can you check that it is running the way you intended: using ssts & ghg forcings from time-varying, historical files?
- What year is the model running?

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
<br>

        
**How do I start my run using initialization datasets from a previous run?**

- Look at chapter `Modifify the run type`. In that chapter, you learned different ways to start up the model, and how to use initial conditions/restart files from a different case to start up a hybrid case. Use that method for the current exercise. 

- Data from spun-up model runs can be found in inputdata (DIN_LOC_ROOT) . 
    
**How do I start my run in 1850?**

- Look at the description of the xml variable `RUN_STARTDATE`
    
    
**How do I output 3 hourly instantaneous variables?**

- Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 
- For more information, look at the chapter: <br>
**NAMELIST MODIFICATIONS** -> **Customize CAM output**

**I am getting an error: ``This compset and grid combination is untested in CESM``**

- overide this error by adding ``--run-unsupported`` to the create_newcase command

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 
 
**# Set environment variables** 

Set environment variables with the commands:

**tcsh**
```
set CASENAME=fhist.1850
set CASEDIR=/glade/u/home/$USER/cases/$CASENAME
set RUNDIR=/glade/derecho/scratch/$USER/$CASENAME/run
set COMPSET=FHIST
set RESOLUTION=f19_f19_mg17
```
**bash**
```
export CASENAME=fhist.1850
export CASEDIR=/glade/u/home/$USER/cases/$CASENAME
export RUNDIR=/glade/derecho/scratch/$USER/$CASENAME/run
export COMPSET=FHIST
export RESOLUTION=f19_f19_mg17
```
    
**# Create a new case**

Create a new case with the command ``create_newcase``:
```
cd /glade/u/home/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case $CASEDIR --res $RESOLUTION --compset $COMPSET --run-unsupported
```

**# Change the job queue and account number**

If needed, change ``job queue`` and ``account number``.<br>
For instance, to run in the queue ``tutorial`` and the project number ``UESM0014``
```  
cd $CASEDIR
./xmlchange JOB_QUEUE=tutorial,PROJECT=UESM0014 --force
```
This step can be redone at anytime in the process. 

**# Setup**

Invoke ``case.setup`` with the command:
```    
cd $CASEDIR
./case.setup    
```  

**# Customize namelists**

Edit the file ``user_nl_cam`` and add the lines:
```
nhtfrq(2) = -3
mfilt(2) = 240
fincl2 = 'TS:I','PS:I', 'U850:I','T850:I','PRECT:I','LHFLX:I','SHFLX:I','FLNT:I','FLNS:I'
```
You can do this with a text editor. Alternatively, you can use the echo command:
```
echo "nhtfrq(2) = -3">> user_nl_cam    
echo "mfilt(2) = 240">> user_nl_cam
echo "fincl2 = 'TS:I','PS:I', 'U850:I','T850:I','PRECT:I','LHFLX:I','SHFLX:I','FLNT:I','FLNS:I'">> user_nl_cam
echo "">> user_nl_cam    
```

You build the namelists with the command:
```
./preview_namelists
```
This step is optional as the script ``preview_namelists`` is automatically called by ``case.build`` and ``case.submit``. But it is nice to check that your changes made their way into:
```
$CASEDIR/CaseDocs/atm_in
```

**# change starting date and ref case**

```   
./xmlchange RUN_STARTDATE=1850-01-01  
./xmlchange RUN_REFCASE=b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001
./xmlchange RUN_REFDATE=0321-01-01
```

**# Set run length**

If needed, change the ``run length``. If you want to run 5 days, you don't have to do this, as 5 days is the default. 
```   
./xmlchange STOP_N=5,STOP_OPTION=ndays
```


**# Build and submit**:
```
qcmd -- ./case.build
./case.submit
```
------------



**# Check your solution**

When the run is completed, look at the history files into the archive directory.  
    
(1) Check that your archive directory on derecho (The path will be different on other machines): 
```
cd /glade/derecho/scratch/$USER/archive/$CASENAME/atm/hist
ls 
```

As your run is only 5-day, there should be no monthly file (``h0``)

Notice that the start year is `1850`.

(2) Look at the contents of the ``h1`` files using ``ncdump``.

- The file should contain the instantaneous output in the file ``h1`` for the variables:
```
        float FLNS(time, lat, lon) ;
                FLNS:Sampling_Sequence = "rad_lwsw" ;
                FLNS:units = "W/m2" ;
                FLNS:long_name = "Net longwave flux at surface" ;
        float FLNT(time, lat, lon) ;
                FLNT:Sampling_Sequence = "rad_lwsw" ;
                FLNT:units = "W/m2" ;
                FLNT:long_name = "Net longwave flux at top of model" ;
        float LHFLX(time, lat, lon) ;
                LHFLX:units = "W/m2" ;
                LHFLX:long_name = "Surface latent heat flux" ;
        float PRECT(time, lat, lon) ;
                PRECT:units = "m/s" ;
                PRECT:long_name = "Total (convective and large-scale) precipitation rate (liq + ice)" ;
        float PS(time, lat, lon) ;
                PS:units = "Pa" ;
                PS:long_name = "Surface pressure" ;
        float SHFLX(time, lat, lon) ;
                SHFLX:units = "W/m2" ;
                SHFLX:long_name = "Surface sensible heat flux" ;
        float T850(time, lat, lon) ;
                T850:units = "K" ;
                T850:long_name = "Temperature at 850 mbar pressure surface" ;
        float TS(time, lat, lon) ;
                TS:units = "K" ;
                TS:long_name = "Surface temperature (radiative)" ;
        float U850(time, lat, lon) ;
                U850:units = "m/s" ;
                U850:long_name = "Zonal wind at 850 mbar pressure surface" ;
```
Note that these variables have no ``cell_methods`` attribute becasue the output is instantaneous. 

</details>
</div>

